<a href="https://colab.research.google.com/github/Janina712/MLTSA22_JBoecher/blob/main/Other/3.%20Match_MLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Import & Set-Up**

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned


In [4]:
reading = pd.read_excel("reading_TextGrid_comb_BG_loop.xlsx")
interview = pd.read_excel("interview_TextGrid_comb_BG_loop.xlsx")

In [5]:
IDs = ['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD',  'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG']

In [6]:
subj_col = pd.DataFrame(['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG'], columns = ["ID"])

In [7]:
control_IDs = ['C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG']

# **Define Functions**

**1. Assign Participant Group**

In [8]:
def assign_group(df):
  group = pd.DataFrame(index = range(len(df)),columns=["Group"])

  for i in range(0,len(df)):
    if "_" in df["ID"][i]:
      group["Group"][i] = "Control"
    else:
      group["Group"][i] = "PWS"
  
  df_out = pd.concat([ group, df], axis=1)
  df_out = df_out[df_out.Type != "silence"]
  df_out.index = range(len(df_out.index))
  df_out.drop(['Unnamed: 0'], axis=1 , inplace = True)
  return(df_out)

**2. Count Vowels**

In [9]:
def count_vowels(df):  
  df_vowels = df[df["Type"]  == "vowel"] 
  df_vowels.index = range(len(df_vowels.index))

  syll_col = pd.DataFrame()  ## initialize group-level dataframe
  for ID in IDs: ## loop over participnts
    syll_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
    subset_sounds = df[df["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
    subset_sounds.index = range(len(subset_sounds.index)) # reset index
    subset_vowels = subset_sounds[subset_sounds["Type"] == "vowel"]  # get subset of sound dataframe that corresponds to current participant
    subset_vowels.index = range(len(subset_vowels.index)) # reset index
    syll = subset_vowels["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
    syll.index = range(len(syll.index)) # reset index
    for a in range (0,len(syll)): # go through all breath groups that this participant produced
      syll_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
      syll_current_BG = pd.DataFrame(np.repeat(syll.iloc[a], syll.iloc[a], axis=0)) #replicate the sum sum times
      syll_current_ID = syll_current_ID.append([syll_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe 
    syll_col = syll_col.append([syll_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

  df_vowels = pd.concat([df_vowels, syll_col], axis=1)
  #df_vowels.drop(['Syllables'], axis=1 , inplace = True)
  df_vowels.rename(columns = {'Syllables':'Unmached_Vowels'}, inplace = True)
  df_vowels.rename(columns = {0:'Syllables'}, inplace = True) # rename new column
  pre_df_vowel_avg = df_vowels.groupby("Group").mean()    ########### average counting 13 13 times

  return(df_vowels, pre_df_vowel_avg)

**3. Count Consonants**

In [10]:
def count_consonants(df):  
  df_consonants = df[df["Type"]  == "consonant"] 
  df_consonants.index = range(len(df_consonants.index))

  con_col = pd.DataFrame()  ## initialize group-level dataframe
  for ID in IDs: ## loop over participnts
    con_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
    subset_sounds = df[df["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
    subset_sounds.index = range(len(subset_sounds.index)) # reset index
    subset_cons = subset_sounds[subset_sounds["Type"] == "consonant"]  # get subset of sound dataframe that corresponds to current participant
    subset_cons.index = range(len(subset_cons.index)) # reset index
    con = subset_cons["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
    con.index = range(len(con.index)) # reset index
    for a in range (0,len(con)): # go through all breath groups that this participant produced
      con_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
      con_current_BG = pd.DataFrame(np.repeat(con.iloc[a], con.iloc[a], axis=0)) #replicate the sum sum times
      con_current_ID = con_current_ID.append([con_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe 
    con_col = con_col.append([con_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

  df_consonants = pd.concat([df_consonants, con_col], axis=1)
  #df_consonants.drop(['Consonants'], axis=1 , inplace = True)
  df_consonants.rename(columns = {'Consonants':'Unmatched_Cons'}, inplace = True)
  df_consonants.rename(columns = {0:'Consonants'}, inplace = True) # rename new column
  pre_df_consonant_avg = df_consonants.groupby("Group").mean()    ########### average counting 13 13 times

  return(df_consonants, pre_df_consonant_avg)

**4. Average Vowel Count by Participant**

In [11]:
def participant_vowel_avg(df):  
  group_col = pd.DataFrame(index = range(len(subj_col)),columns=["Group"])
  for i in range(0,len(group_col)):
    if "_" in subj_col["ID"][i]:
      group_col["Group"][i] = "Control"
    else:
      group_col["Group"][i] = "PWS"
  
  n = -1
  avg_col = pd.DataFrame(index = range(len(subj_col)),columns=["Syllables"])
  for ID in IDs: ## loop over participnts
    n = n + 1
    subset_BGs = df[df["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
    subset_BGs.index = range(len(subset_BGs.index)) # reset index
    BG_avg = subset_BGs.groupby("Breath.Group").mean()
    subj_avg = BG_avg["Syllables"].mean()
    avg_col["Syllables"][n] = subj_avg

  df_participant_vowel_avg = pd.concat([group_col, subj_col, avg_col], axis=1)

  return(df_participant_vowel_avg)

**5. Average Consonant Count by Participant**

In [12]:
def participant_consonant_avg(df):  
  group_col = pd.DataFrame(index = range(len(subj_col)),columns=["Group"])
  for i in range(0,len(group_col)):
    if "_" in subj_col["ID"][i]:
      group_col["Group"][i] = "Control"
    else:
      group_col["Group"][i] = "PWS"
  
  n = -1
  avg_col = pd.DataFrame(index = range(len(subj_col)),columns=["Consonants"])
  for ID in IDs: ## loop over participnts
    n = n + 1
    subset_BGs = df[df["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
    subset_BGs.index = range(len(subset_BGs.index)) # reset index
    BG_avg = subset_BGs.groupby("Breath.Group").mean()
    subj_avg = BG_avg["Consonants"].mean()
    avg_col["Consonants"][n] = subj_avg

  df_participant_cons_avg = pd.concat([group_col, subj_col, avg_col], axis=1)

  return(df_participant_cons_avg)

**6. Vowel to Consonant Ratio**

In [13]:
def get_Ratio(condition, df_vowels_pws, df_consonants_pws, df_vowels_control, df_consonants_control):
  vowel_perc_p = df_vowels_pws[df_vowels_pws["Group"] == "PWS"]
  consonant_perc_p = df_consonants_pws[df_consonants_pws["Group"] == "PWS"]
  vowel_perc_c = df_vowels_control[df_vowels_control["Group"] == "Control"]
  consonant_perc_c = df_consonants_control[df_consonants_control["Group"] == "Control"]

  vowel_ratio_p = len(vowel_perc_p)/(len(vowel_perc_p) + len(consonant_perc_p))
  vowel_ratio_c = len(vowel_perc_c)/(len(vowel_perc_c) + len(consonant_perc_c))

  string = (f"Before matching, PWS have {round(vowel_ratio_p,4)*100}% vowels and Control have {round(vowel_ratio_c,4)*100}% vowels in their {condition} utterances. This ratio is a measure of number of vowels.")
  print(string)
  return(string)

**7. Compare Consonant and Vowel Counts Across Participant Groups (PWS vs. Control)**

In [14]:
def compare_groups(df_vowels, df_consonants):
  control_v = (df_vowels.groupby("ID").mean()["Syllables"][18:30]).mean()
  pws_v = (df_vowels.groupby("ID").mean()["Syllables"][0:18]).mean()
  difference_v = control_v - pws_v

  control_c = (df_consonants.groupby("ID").mean()["Consonants"][18:30]).mean()
  pws_c = (df_consonants.groupby("ID").mean()["Consonants"][0:18]).mean()
  difference_c = control_c - pws_c

  string1 = (f"PWS produced on average {round(pws_v,2)} syllables per utterance, while control participants produced {round(control_v,2)} syllables on average.")
  string2 = (f"This means that on average control participants produced {round(difference_v,2)} syllables more per utterance.")
  string3 = (f"\nPWS produced on average {round(pws_c,2)} consonants per utterance, while control participants produced {round(control_c,2)} consonants on average.")
  string4 = (f"This means that on average control participants produced {round(difference_c,2)} consonants more per utterance.")

  return(string1, string2, string3, string4, pws_v, pws_c, difference_v, difference_c)

**8. Match Number of Vowels Per Utterance**

In [15]:
def match_vowels(df_vowels, pws_v, cut_v):
  cut_v = math.ceil(cut_v) ##+ 5                                                           
  syll_col_matched = pd.DataFrame()  ## initialize group-level dataframe              
  participant = pd.DataFrame()  ## initialize participant-level dataframe
  df_control = df_vowels[df_vowels["Group"]  == "Control"] 
  df_control.index = range(len(df_control.index)) ## group
  for ID in control_IDs:
    df_control_ID = df_control[df_control["ID"]  == ID] 
    df_control_ID.index = range(len(df_control_ID.index))  ### person
    BGs = df_control_ID["Breath.Group"].unique()
    for i in range(0,len(BGs)):
      df_control_ID_BG = df_control_ID[df_control_ID["Breath.Group"]  == i] 
      df_control_ID_BG.index = range(len(df_control_ID_BG.index)) ## BG
      if len(df_control_ID_BG) >= cut_v:
        df_control_ID_BG.drop(df_control_ID_BG.tail(cut_v).index, inplace = True) 
        participant = participant.append([df_control_ID_BG], ignore_index = True)
      else:  
        participant = participant.append([df_control_ID_BG], ignore_index = True)
  syll_col_matched = syll_col_matched.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe

  return(syll_col_matched)

**9. Match Number of Consonants Per Utterance**

In [16]:
def match_consonants(df_consonants, pws_c, cut_c):
  cut_c = math.ceil(cut_c) ##+ 4                                                       
  cons_col_matched = pd.DataFrame()  ## initialize group-level dataframe         
  participant = pd.DataFrame()  ## initialize participant-level dataframe
  df_control = df_consonants[df_consonants["Group"]  == "Control"] 
  df_control.index = range(len(df_control.index)) ## group
  for ID in control_IDs:
    df_control_ID = df_control[df_control["ID"]  == ID] 
    df_control_ID.index = range(len(df_control_ID.index))  ### person
    BGs = df_control_ID["Breath.Group"].unique()
    for i in range(0,len(BGs)):
      df_control_ID_BG = df_control_ID[df_control_ID["Breath.Group"]  == i] 
      df_control_ID_BG.index = range(len(df_control_ID_BG.index)) ## BG
      if len(df_control_ID_BG) >= cut_c:
        df_control_ID_BG.drop(df_control_ID_BG.tail(cut_c).index, inplace = True) 
        participant = participant.append([df_control_ID_BG], ignore_index = True)
      else:  
        participant = participant.append([df_control_ID_BG], ignore_index = True)
  cons_col_matched = cons_col_matched.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe
  return(cons_col_matched)

**10. Test if Remaining Vowel Difference Between Groups is Statistically Significant**

In [17]:
def ttest_vowelDifference(matched_participant_df, cut_v): 
  cut_v = round(cut_v)
  ttest_control = matched_participant_df[18:30]  
  ttest_pws = matched_participant_df[0:18]   

  control_v =  matched_participant_df[18:30].mean()[0]
  pws_v =  matched_participant_df[0:18].mean()[0]

  difference_v_matched = control_v - pws_v

  df_v_matched_test = stats.ttest_ind(ttest_control['Syllables'],
                        ttest_pws['Syllables'])
  if df_v_matched_test[1] >= 0.05:
    level = "insignificant"
  else:
    level = "significant"
  string11 = (f"Control utterances that were longer than the mean average length of PWS utterances were shortened. \nAfter matching, PWS had {round(pws_v,2)} vowels per utterance, while control participants had {round(control_v,2)} per utterance. \nThe difference is reduced to {abs(round(difference_v_matched,2))}, which is statistically {level} (p = {round(df_v_matched_test[1],2)}).")

  return(string11)

**11. Test if Remaining Consonant Difference Between Groups is Statistically Significant**

In [18]:
def ttest_conDifference(matched_participant_df, cut_c):  
  cut_c = round(cut_c)
  ttest_control = matched_participant_df[18:30]
  ttest_pws = matched_participant_df[0:18]  

  control_c =  matched_participant_df[18:30].mean()[0]
  pws_c =  matched_participant_df[0:18].mean()[0]

  difference_c_matched = control_c - pws_c

  df_c_matched_test = stats.ttest_ind(ttest_control['Consonants'],
                        ttest_pws['Consonants'])
  if df_c_matched_test[1] >= 0.05:
    level = "insignificant"
  else:
    level = "significant"
  string11 = (f"Control utterances that were longer than the mean average length of PWS utterances, were shortened by {round(cut_c,2)} consonants. \nAfter matching, PWS had {round(pws_c,2)} consonants per utterance, while control participants had {round(control_c,2)} per utterance. \nThe difference is reduced to {abs(round(difference_c_matched,2))}, which is statistically {level} (p = {round(df_c_matched_test[1],2)}).")

  return(string11)

# **Conduct Matching Analysis using Functions**

In [19]:
# add new column to dataframe that denotes participant's group membership (PWS vs. Control)
interview = assign_group(interview)
reading = assign_group(reading)

In [20]:
# count vowels per utterance 
# output 1: df of size "interview", with an additional #vowels per utterance for each row (redundant for BG)
# output 2: df of size "len(Group)" (2), that contains average # vowels per breath group for PWS group vs. Control group 
[interview_vowels, pre_interview_vowel_avg]  = count_vowels(interview)
[reading_vowels, pre_reading_vowel_avg] = count_vowels(reading)

In [21]:
# count consonants per utterance 
# output 1: df of size "interview", with an additional #consonants per utterance for each row (redundant for BG)
# output 2: df of size "len(Group)" (2), that contains average # consonants per breath group for PWS group vs. Control group 
[interview_consonants, pre_interview_consonant_avg]  = count_consonants(interview)
[reading_consonants, pre_reading_consonants_avg] = count_consonants(reading)

In [22]:
#average # vowels per breath group for each participant 
interview_participant_vowel_avg = participant_vowel_avg(interview_vowels)
reading_participant_vowel_avg = participant_vowel_avg(reading_vowels)

In [23]:
#average # consonants per breath group for each participant 
interview_participant_cons_avg = participant_consonant_avg(interview_consonants)
reading_participant_cons_avg = participant_consonant_avg(reading_consonants)

In [24]:
# Print a string that contains the percentage of vowels in dataframe
pre_matching_reading  = get_Ratio("reading", reading_vowels, reading_consonants, reading_vowels, reading_consonants)
pre_matching_interview  = get_Ratio("interview", interview_vowels, interview_consonants, interview_vowels, interview_consonants)

Before matching, PWS have 39.43% vowels and Control have 39.45% vowels in their reading utterances. This ratio is a measure of number of vowels.
Before matching, PWS have 40.37% vowels and Control have 39.07% vowels in their interview utterances. This ratio is a measure of number of vowels.


In [25]:
# compare group averages of number of vowels per utterance and number of consonants per utterance 
# interview
string1, string2, string3, string4, pws_v_inter, pws_c_inter, difference_v_inter, difference_c_inter = compare_groups(interview_participant_vowel_avg, interview_participant_cons_avg)

print(string1)
print(string2)
print(string3)
print(string4)

PWS produced on average 7.81 syllables per utterance, while control participants produced 9.7 syllables on average.
This means that on average control participants produced 1.9 syllables more per utterance.

PWS produced on average 11.49 consonants per utterance, while control participants produced 15.17 consonants on average.
This means that on average control participants produced 3.68 consonants more per utterance.


In [26]:
# compare group averages of number of vowels per utterance and number of consonants per utterance 
# reading
string5, string6, string7, string8, pws_v_read, pws_c_read, difference_v_read, difference_c_read  = compare_groups(reading_participant_vowel_avg, reading_participant_cons_avg)

print(string5)
print(string6)
print(string7)
print(string8)

PWS produced on average 8.3 syllables per utterance, while control participants produced 11.4 syllables on average.
This means that on average control participants produced 3.1 syllables more per utterance.

PWS produced on average 12.75 consonants per utterance, while control participants produced 17.49 consonants on average.
This means that on average control participants produced 4.74 consonants more per utterance.


In [27]:
# match number of vowels per utterance by cutting long control utterances by the difference in average vowel number between groups
matched_vowels_read = match_vowels(reading_vowels,pws_v_read, difference_v_read)
matched_vowels_inter = match_vowels(interview_vowels, pws_v_inter, difference_v_inter)

In [28]:
# match number of consonants per utterance by cutting long control utterances by the difference in average consonant number between groups
matched_consonants_read = match_vowels(reading_consonants,pws_c_read, difference_c_read)
matched_consonants_inter = match_vowels(interview_consonants, pws_c_inter, difference_c_inter)

In [29]:
# count vowels per utterance after matching
# output 1: df of size "interview minus cut vowel rows", with an updated #vowels per utterance for each row (redundant for BG)
# output 2: df of size "len(Group)" (2), that contains average # vowels per breath group for PWS group vs. Control group after matching
[post_interview_vowels, post_interview_vowel_avg]  = count_vowels(matched_vowels_inter)
[post_reading_vowels, post_reading_vowel_avg]  = count_vowels(matched_vowels_read)

In [30]:
# average # vowels per breath group for each participant after matching
post_interview_participant_vowels_avg = participant_vowel_avg(post_interview_vowels)
post_reading_participant_vowels_avg = participant_vowel_avg(post_reading_vowels)
# only control group has been changed
post_interview_participant_vowels_avg = post_interview_participant_vowels_avg[18:30]
post_reading_participant_vowels_avg = post_reading_participant_vowels_avg[18:30]
# combine with pre_matching (unchanged pws)avgs
post_interview_participant_vowels_avg = interview_participant_vowel_avg[0:18].append(post_interview_participant_vowels_avg)
post_reading_participant_vowels_avg = reading_participant_vowel_avg[0:18].append(post_reading_participant_vowels_avg)

In [31]:
# count consonants per utterance after matching
# output 1: df of size "interview minus cut consonant rows", with an updated #consonants per utterance for each row (redundant for BG)
# output 2: df of size "len(Group)" (2), that contains average # consonants per breath group for PWS group vs. Control group after matching 
[post_interview_consonants, post_interview_consonant_avg]  = count_consonants(matched_consonants_inter)
[post_reading_consonants, post_reading_consonant_avg]  = count_consonants(matched_consonants_read)

In [32]:
#average # consonants per breath group for each participant after matching
post_interview_participant_cons_avg = participant_consonant_avg(post_interview_consonants)
post_reading_participant_cons_avg = participant_consonant_avg(post_reading_consonants)
# only control group has been changed
post_interview_participant_cons_avg = post_interview_participant_cons_avg[18:30]
post_reading_participant_cons_avg = post_reading_participant_cons_avg[18:30]
# combine with pre_matching (unchanged pws)avgs
post_interview_participant_cons_avg = interview_participant_cons_avg[0:18].append(post_interview_participant_cons_avg)
post_reading_participant_cons_avg = reading_participant_cons_avg[0:18].append(post_reading_participant_cons_avg)

In [33]:
# Print a string that contains the percentage of vowels in dataframe
post_matching_reading  = get_Ratio("reading", reading_vowels, reading_consonants, post_reading_vowels, post_reading_consonants)
post_matching_interview  = get_Ratio("interview", interview_vowels, interview_consonants, post_interview_vowels, post_interview_consonants)

Before matching, PWS have 39.43% vowels and Control have 37.3% vowels in their reading utterances. This ratio is a measure of number of vowels.
Before matching, PWS have 40.37% vowels and Control have 40.910000000000004% vowels in their interview utterances. This ratio is a measure of number of vowels.


In [34]:
# compare group averages of number of vowels per utterance and number of consonants per utterance
[string1, string2, string3, string4, pws_v_inter, pws_c_inter, difference_v_inter, difference_c_inter] = compare_groups(post_interview_participant_vowels_avg, post_interview_participant_cons_avg)

print(string1)
print(string2)
print(string3)
print(string4)

PWS produced on average 7.81 syllables per utterance, while control participants produced 7.69 syllables on average.
This means that on average control participants produced -0.12 syllables more per utterance.

PWS produced on average 11.49 consonants per utterance, while control participants produced 11.25 consonants on average.
This means that on average control participants produced -0.23 consonants more per utterance.


In [35]:
[string5, string6, string7, string8, control_v_read, pws_v_read, difference_v_read, difference_c_read] = compare_groups(post_reading_participant_vowels_avg, post_reading_participant_cons_avg)
print(string5)
print(string6)
print(string7)
print(string8)

PWS produced on average 8.3 syllables per utterance, while control participants produced 7.94 syllables on average.
This means that on average control participants produced -0.36 syllables more per utterance.

PWS produced on average 12.75 consonants per utterance, while control participants produced 12.77 consonants on average.
This means that on average control participants produced 0.02 consonants more per utterance.


In [36]:
# test if average number of vowels per breathgroup is still significantly different between groups after matching
string1 = ttest_vowelDifference(post_interview_participant_vowels_avg, difference_v_inter)
print(string1)


Control utterances that were longer than the mean average length of PWS utterances were shortened. 
After matching, PWS had 7.81 vowels per utterance, while control participants had 7.69 per utterance. 
The difference is reduced to 0.12, which is statistically insignificant (p = 0.84).


In [37]:
string2 = ttest_vowelDifference(post_reading_participant_vowels_avg, difference_v_read)
print(string2)

Control utterances that were longer than the mean average length of PWS utterances were shortened. 
After matching, PWS had 8.3 vowels per utterance, while control participants had 7.94 per utterance. 
The difference is reduced to 0.36, which is statistically insignificant (p = 0.45).


In [38]:
# test if average number of consonants per breathgroup is still significantly different between groups after matching
string1 = ttest_conDifference(post_interview_participant_cons_avg, difference_c_inter)
print(string1)

Control utterances that were longer than the mean average length of PWS utterances, were shortened by 0 consonants. 
After matching, PWS had 11.49 consonants per utterance, while control participants had 11.25 per utterance. 
The difference is reduced to 0.23, which is statistically insignificant (p = 0.79).


In [39]:
string2 = ttest_conDifference(post_reading_participant_cons_avg, difference_c_read)
print(string2)

Control utterances that were longer than the mean average length of PWS utterances, were shortened by 0 consonants. 
After matching, PWS had 12.75 consonants per utterance, while control participants had 12.77 per utterance. 
The difference is reduced to 0.02, which is statistically insignificant (p = 0.98).


# **Export Results**

**1. Create Output DataFrames**

In [ ]:
# reading vowels
pws = reading_vowels[reading_vowels["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
post_reading_vowels.drop(["Unmached_Vowels"],axis=1 , inplace = True)
reading_vowels_matched = pws.append(post_reading_vowels)

In [ ]:
# interview vowels
pws = interview_vowels[interview_vowels["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
post_interview_vowels.drop(["Unmached_Vowels"],axis=1 , inplace = True)
interview_vowels_matched = pws.append(post_interview_vowels)

In [ ]:
#reading consonants
pws = reading_consonants[reading_consonants["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
post_reading_consonants.drop(["Unmatched_Cons"],axis=1 , inplace = True)
reading_consonants_matched = pws.append(post_reading_consonants)

In [ ]:
# interview consonants
pws = interview_consonants[interview_consonants["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
post_interview_consonants.drop(["Unmatched_Cons"],axis=1 , inplace = True)
interview_consonants_matched = pws.append(post_interview_consonants)

**2. Create Directory and Save**

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

In [ ]:
dir = "3.MLU_Matched"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched/

In [ ]:
interview_vowels_matched.to_excel("interview_TextGrid_comb_BG_vowelsMatched.xlsx")
interview_consonants_matched.to_excel("interview_TextGrid_comb_BG_consonantsMatched.xlsx")
reading_vowels_matched.to_excel("reading_TextGrid_comb_BG_vowelsMatched.xlsx")
reading_consonants_matched.to_excel("reading_TextGrid_comb_BG_consonantsMatched.xlsx")

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

# **Match Fluent vs. Disfluent**

**Divide dataframe by groups**

In [59]:
reading_vowels_pws = reading_vowels[reading_vowels["Group"] == "PWS"]
reading_vowels_pws.index = range(len(reading_vowels_pws.index))
reading_vowels_control = reading_vowels[reading_vowels["Group"] == "Control"]
reading_vowels_control.index = range(len(reading_vowels_control.index))

In [61]:
reading_consonants_pws = reading_consonants[reading_consonants["Group"] == "PWS"]
reading_consonants_pws.index = range(len(reading_consonants_pws.index))
reading_consonants_control = reading_consonants[reading_consonants["Group"] == "Control"]
reading_consonants_control.index = range(len(reading_consonants_control.index))

**Get means for fluent and disfluent utterances for each group for reading**

In [62]:
# reading, pws
fluent_v_r_p = reading_vowels_pws.groupby(["FluencyStatus"]).mean()["Syllables"][1]
disfluent_v_r_p = reading_vowels_pws.groupby(["FluencyStatus"]).mean()["Syllables"][0]
fluent_c_r_p = reading_consonants_pws.groupby(["FluencyStatus"]).mean()["Consonants"][1]
disfluent_c_r_p = reading_consonants_pws.groupby(["FluencyStatus"]).mean()["Consonants"][0]

In [63]:
# reading, control
fluent_v_r_c = reading_vowels_control.groupby(["FluencyStatus"]).mean()["Syllables"][1]
disfluent_v_r_c = reading_vowels_control.groupby(["FluencyStatus"]).mean()["Syllables"][0]
fluent_c_r_c = reading_consonants_control.groupby(["FluencyStatus"]).mean()["Consonants"][1]
disfluent_c_r_c = reading_consonants_control.groupby(["FluencyStatus"]).mean()["Consonants"][0]

**Divide dataframe by groups**

In [66]:
# interview vowel groups
interview_vowels_pws = interview_vowels[interview_vowels["Group"] == "PWS"]
interview_vowels_pws.index = range(len(interview_vowels_pws.index))
interview_vowels_control = interview_vowels[interview_vowels["Group"] == "Control"]
interview_vowels_control.index = range(len(interview_vowels_control.index))

In [64]:
# interview consonant groups
interview_consonants_pws = interview_consonants[interview_consonants["Group"] == "PWS"]
interview_consonants_pws.index = range(len(interview_consonants_pws.index))
interview_consonants_control = interview_consonants[interview_consonants["Group"] == "Control"]
interview_consonants_control.index = range(len(interview_consonants_control.index))

**Get means for fluent and disfluent utterances for each group for interview**

In [67]:
# interview, pws
fluent_v_i_p = interview_vowels_pws.groupby(["FluencyStatus"]).mean()["Syllables"][1]
disfluent_v_i_p = interview_vowels_pws.groupby(["FluencyStatus"]).mean()["Syllables"][0]
fluent_c_i_p = interview_consonants_pws.groupby(["FluencyStatus"]).mean()["Consonants"][1]
disfluent_c_i_p = interview_consonants_pws.groupby(["FluencyStatus"]).mean()["Consonants"][0]

In [68]:
# interview, control
fluent_v_i_c = interview_vowels_control.groupby(["FluencyStatus"]).mean()["Syllables"][1]
disfluent_v_i_c = interview_vowels_control.groupby(["FluencyStatus"]).mean()["Syllables"][0]
fluent_c_i_c = interview_consonants_control.groupby(["FluencyStatus"]).mean()["Consonants"][1]
disfluent_c_i_c = interview_consonants_control.groupby(["FluencyStatus"]).mean()["Consonants"][0]

**Compare the Means**

In [78]:
string = f"In the reading condition, disfluent utterances produced by PWS contained {round(disfluent_v_r_p,2)} vowels and {round(disfluent_c_r_p,2)} consonants, while fluent utterances contained {round(fluent_v_r_p,2)} vowels and {round(fluent_c_r_p,2)} consonants."
string1 = f"The difference between vowels is {round(fluent_v_r_p - disfluent_v_r_p,2)} and the difference between consonants is {round(fluent_c_r_p - disfluent_c_r_p,2)}. "
string2 = f"\nBy contrast, disfluent utterances produced by control participants contained {round(disfluent_v_r_c,2)} vowels and {round(disfluent_c_r_c,2)} consonants, while fluent utterances contained {round(fluent_v_r_c,2)} vowels and {round(fluent_c_r_c,2)} consonants."
string3 = f"The difference between vowels is {round(fluent_v_r_c - disfluent_v_r_c,2)} and the difference between consonants is {round(fluent_c_r_c - disfluent_c_r_c,2)}. "

In [79]:
print(string)
print(string1)
print(string2)
print(string3)

In the reading condition, disfluent utterances produced by PWS contained 8.37 vowels and 13.24 consonants, while fluent utterances contained 11.89 vowels and 18.45 consonants.
The difference between vowels is 3.51 and the difference between consonants is 5.21. 

By contrast, disfluent utterances produced by control participants contained 9.99 vowels and 18.1 consonants, while fluent utterances contained 14.09 vowels and 21.4 consonants.
The difference between vowels is 4.11 and the difference between consonants is 3.3. 


In [80]:
string = f"In the interview condition, disfluent utterances produced by PWS contained {round(disfluent_v_i_p,2)} vowels and {round(disfluent_c_i_p,2)} consonants, while fluent utterances contained {round(fluent_v_i_p,2)} vowels and {round(fluent_c_i_p,2)} consonants."
string1 = f"The difference between vowels is {round(fluent_v_i_p - disfluent_v_i_p,2)} and the difference between consonants is {round(fluent_c_i_p - disfluent_c_i_p,2)}. "
string2 = f"\nBy contrast, disfluent utterances produced by control participants contained {round(disfluent_v_i_c,2)} vowels and {round(disfluent_c_i_c,2)} consonants, while fluent utterances contained {round(fluent_v_i_c,2)} vowels and {round(fluent_c_i_c,2)} consonants."
string3 = f"The difference between vowels is {round(fluent_v_i_c - disfluent_v_i_c,2)} and the difference between consonants is {round(fluent_c_i_c - disfluent_c_i_c,2)}. "

In [81]:
print(string)
print(string1)
print(string2)
print(string3)

In the interview condition, disfluent utterances produced by PWS contained 7.84 vowels and 11.57 consonants, while fluent utterances contained 11.49 vowels and 17.17 consonants.
The difference between vowels is 3.64 and the difference between consonants is 5.59. 

By contrast, disfluent utterances produced by control participants contained 9.87 vowels and 15.33 consonants, while fluent utterances contained 12.14 vowels and 18.99 consonants.
The difference between vowels is 2.27 and the difference between consonants is 3.66. 


**Match fluent vowels and consonants to match the length of disfluent vowels and consonants**

**Reading**

In [ ]:
# vowels

In [131]:
fluent_vowel_subset = reading_vowels_pws[reading_vowels_pws["FluencyStatus"] == "Fluent"]
fluent_vowel_subset.index = range(len(fluent_vowel_subset.index))
difference_v_read_pws = fluent_v_r_p - disfluent_v_r_p

In [132]:
cut_v = math.ceil(difference_v_read_pws)                                                         
matched_r_v_p = pd.DataFrame()  ## initialize group-level dataframe              
participant = pd.DataFrame()  ## initialize participant-level dataframe
for ID in IDs:
  subset_ID = fluent_vowel_subset[fluent_vowel_subset["ID"]  == ID] 
  subset_ID.index = range(len(subset_ID.index))  ### person
  BGs = subset_ID["Breath.Group"].unique()
  for i in range(0,len(BGs)):
    subset_ID_BG = subset_ID[subset_ID["Breath.Group"]  == i] 
    subset_ID_BG.index = range(len(subset_ID_BG.index)) ## BG
    if len(subset_ID_BG) >= cut_v:
      subset_ID_BG.drop(subset_ID_BG.tail(cut_v).index, inplace = True) 
      participant = participant.append([subset_ID_BG], ignore_index = True)
    else:  
      participant = participant.append([subset_ID_BG], ignore_index = True)
matched_r_v_p = matched_r_v_p.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [ ]:
# consonants 

In [133]:
fluent_consonant_subset = reading_consonants_pws[reading_consonants_pws["FluencyStatus"] == "Fluent"]
fluent_consonant_subset.index = range(len(fluent_consonant_subset.index))
difference_c_read_pws = fluent_c_r_p - disfluent_c_r_p

In [134]:
cut_v = math.ceil(difference_c_read_pws)                                                         
matched_r_c_p = pd.DataFrame()  ## initialize group-level dataframe              
participant = pd.DataFrame()  ## initialize participant-level dataframe
for ID in IDs:
  subset_ID = fluent_consonant_subset[fluent_consonant_subset["ID"]  == ID] 
  subset_ID.index = range(len(subset_ID.index))  ### person
  BGs = subset_ID["Breath.Group"].unique()
  for i in range(0,len(BGs)):
    subset_ID_BG = subset_ID[subset_ID["Breath.Group"]  == i] 
    subset_ID_BG.index = range(len(subset_ID_BG.index)) ## BG
    if len(subset_ID_BG) >= cut_v:
      subset_ID_BG.drop(subset_ID_BG.tail(cut_v).index, inplace = True) 
      participant = participant.append([subset_ID_BG], ignore_index = True)
    else:  
      participant = participant.append([subset_ID_BG], ignore_index = True)
matched_r_c_p = matched_r_c_p.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [106]:
matched_r_c_p

,Group,ID,Type,Sound,Onset,Offset,Duration,Breath.Group,FluencyStatus,m,Consonants
0,PWS,24fa,consonant,"""S""",0.143,0.263,0.120,1,Fluent,17,9
1,PWS,24fa,consonant,"""HH""",0.343,0.482,0.139,1,Fluent,17,9
2,PWS,24fa,consonant,"""R""",0.573,0.702,0.129,1,Fluent,17,9
3,PWS,24fa,consonant,"""HH""",3.303,3.403,0.100,3,Fluent,24,14
4,PWS,24fa,consonant,"""V""",3.433,3.542,0.109,3,Fluent,24,14
...,...,...,...,...,...,...,...,...,...,...,...
1955,PWS,62m,consonant,"""R""",29.595,29.715,0.120,18,Fluent,21,12
1956,PWS,62m,consonant,"""K""",29.835,29.865,0.030,18,Fluent,21,12
1957,PWS,62m,consonant,"""L""",29.865,29.896,0.031,18,Fluent,21,12
1958,PWS,62m,consonant,"""T""",29.946,30.035,0.089,18,Fluent,21,12


**Interview**

In [ ]:
# vowels

In [108]:
fluent_vowel_subset = interview_vowels_pws[interview_vowels_pws["FluencyStatus"] == "Fluent"]
fluent_vowel_subset.index = range(len(fluent_vowel_subset.index))
difference_v_inter_pws = fluent_v_i_p - disfluent_v_i_p

In [109]:
cut_v = math.ceil(difference_v_inter_pws)                                                         
matched_i_v_p = pd.DataFrame()  ## initialize group-level dataframe              
participant = pd.DataFrame()  ## initialize participant-level dataframe
for ID in IDs:
  subset_ID = fluent_vowel_subset[fluent_vowel_subset["ID"]  == ID] 
  subset_ID.index = range(len(subset_ID.index))  ### person
  BGs = subset_ID["Breath.Group"].unique()
  for i in range(0,len(BGs)):
    subset_ID_BG = subset_ID[subset_ID["Breath.Group"]  == i] 
    subset_ID_BG.index = range(len(subset_ID_BG.index)) ## BG
    if len(subset_ID_BG) >= cut_v:
      subset_ID_BG.drop(subset_ID_BG.tail(cut_v).index, inplace = True) 
      participant = participant.append([subset_ID_BG], ignore_index = True)
    else:  
      participant = participant.append([subset_ID_BG], ignore_index = True)
matched_i_v_p = matched_i_v_p.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [ ]:
# consonants

In [111]:
fluent_consonant_subset = interview_consonants_pws[interview_consonants_pws["FluencyStatus"] == "Fluent"]
fluent_consonant_subset.index = range(len(fluent_consonant_subset.index))
difference_c_inter_pws = fluent_c_i_p - disfluent_c_i_p

In [112]:
cut_v = math.ceil(difference_c_inter_pws)                                                         
matched_i_c_p = pd.DataFrame()  ## initialize group-level dataframe              
participant = pd.DataFrame()  ## initialize participant-level dataframe
for ID in IDs:
  subset_ID = fluent_consonant_subset[fluent_consonant_subset["ID"]  == ID] 
  subset_ID.index = range(len(subset_ID.index))  ### person
  BGs = subset_ID["Breath.Group"].unique()
  for i in range(0,len(BGs)):
    subset_ID_BG = subset_ID[subset_ID["Breath.Group"]  == i] 
    subset_ID_BG.index = range(len(subset_ID_BG.index)) ## BG
    if len(subset_ID_BG) >= cut_v:
      subset_ID_BG.drop(subset_ID_BG.tail(cut_v).index, inplace = True) 
      participant = participant.append([subset_ID_BG], ignore_index = True)
    else:  
      participant = participant.append([subset_ID_BG], ignore_index = True)
matched_i_c_p = matched_i_c_p.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe

**Count new number of Sounds in BG: Reading**

In [153]:
n = -1
avg_col = pd.DataFrame(index = range(len(subj_col[0:18])),columns=["Syllables"])
for ID in IDs: ## loop over participnts
  n = n + 1
  subset_BGs = matched_r_v_p[matched_r_v_p["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_BGs.index = range(len(subset_BGs.index)) # reset index
  BG_avg = subset_BGs.groupby("Breath.Group").mean()
  subj_avg = BG_avg["Syllables"].mean()
  avg_col["Syllables"][n] = subj_avg
participant_vowel_r_p_avg = pd.concat([subj_col[0:18], avg_col], axis=1)

In [150]:
n = -1
avg_col_c = pd.DataFrame(index = range(len(subj_col[0:18])),columns=["Consonants"])
for ID in IDs: ## loop over participnts
  n = n + 1
  subset_BGs = matched_r_c_p[matched_r_c_p["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_BGs.index = range(len(subset_BGs.index)) # reset index
  BG_avg = subset_BGs.groupby("Breath.Group").mean()
  subj_avg_c = BG_avg["Consonants"].mean()
  avg_col_c["Consonants"][n] = subj_avg_c
participant_consonant_r_p_avg = pd.concat([subj_col[0:18], avg_col_c], axis=1)

**Compare the new means**

In [ ]:
# reading

In [155]:
new_vowel = participant_vowel_r_p_avg["Syllables"].mean()

In [156]:
new_cons = participant_consonant_r_p_avg["Consonants"].mean()

In [157]:
string = f"In the reading condition, disfluent utterances produced by PWS contained {round(disfluent_v_r_p,2)} vowels and {round(disfluent_c_r_p,2)} consonants, while fluent utterances contained {round(new_vowel,2)} vowels and {round(new_cons,2)} consonants."
string1 = f"The difference between vowels is {round(fluent_v_r_p - disfluent_v_r_p,2)} and the difference between consonants is {round(fluent_c_r_p - disfluent_c_r_p,2)}. "

In [158]:
print(string)

In the reading condition, disfluent utterances produced by PWS contained 8.37 vowels and 13.24 consonants, while fluent utterances contained 9.41 vowels and 13.88 consonants.


**Count new number of Sounds in BG: Interview**

In [163]:
# interview
n = -1
avg_col = pd.DataFrame(index = range(len(subj_col[0:18])),columns=["Syllables"])
for ID in IDs: ## loop over participnts
  n = n + 1
  subset_BGs = matched_i_v_p[matched_i_v_p["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_BGs.index = range(len(subset_BGs.index)) # reset index
  BG_avg = subset_BGs.groupby("Breath.Group").mean()
  subj_avg = BG_avg["Syllables"].mean()
  avg_col["Syllables"][n] = subj_avg
participant_vowel_i_p_avg = pd.concat([subj_col[0:18], avg_col], axis=1)

In [164]:
n = -1
avg_col_c = pd.DataFrame(index = range(len(subj_col[0:18])),columns=["Consonants"])
for ID in IDs: ## loop over participnts
  n = n + 1
  subset_BGs = matched_i_c_p[matched_i_c_p["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_BGs.index = range(len(subset_BGs.index)) # reset index
  BG_avg = subset_BGs.groupby("Breath.Group").mean()
  subj_avg_c = BG_avg["Consonants"].mean()
  avg_col_c["Consonants"][n] = subj_avg_c
participant_consonant_i_p_avg = pd.concat([subj_col[0:18], avg_col_c], axis=1)

**Compare the new means**

In [165]:
new_vowel = participant_vowel_i_p_avg["Syllables"].mean()
new_cons = participant_consonant_i_p_avg["Consonants"].mean()

In [166]:
string = f"In the interview condition, disfluent utterances produced by PWS contained {round(disfluent_v_i_p,2)} vowels and {round(disfluent_c_i_p,2)} consonants, while fluent utterances contained {round(new_vowel,2)} vowels and {round(new_cons,2)} consonants."
string1 = f"The difference between vowels is {round(fluent_v_r_p - disfluent_v_r_p,2)} and the difference between consonants is {round(fluent_c_r_p - disfluent_c_r_p,2)}. "

In [167]:
print(string)

In the interview condition, disfluent utterances produced by PWS contained 7.84 vowels and 11.57 consonants, while fluent utterances contained 8.92 vowels and 12.91 consonants.


# **Export FluencyStatus Results**

**1. Create output dataframes**

In [175]:
reading_vowels_pws_disfluent = reading_vowels_pws[reading_vowels_pws["FluencyStatus"] == "Disfluent"]
matched_r_v_p = matched_r_v_p.append(reading_vowels_pws_disfluent)
matched_r_v_p.index = range(len(matched_r_v_p.index)) 

In [177]:
reading_consonants_pws_disfluent = reading_consonants_pws[reading_consonants_pws["FluencyStatus"] == "Disfluent"]
matched_r_c_p = matched_r_c_p.append(reading_consonants_pws_disfluent)
matched_r_c_p.index = range(len(matched_r_c_p.index)) 

In [179]:
interview_vowels_pws_disfluent = interview_vowels_pws[interview_vowels_pws["FluencyStatus"] == "Disfluent"]
matched_i_v_p = matched_i_v_p.append(interview_vowels_pws_disfluent)
matched_i_v_p.index = range(len(matched_i_v_p.index)) 

In [181]:
interview_consonants_pws_disfluent = interview_consonants_pws[interview_consonants_pws["FluencyStatus"] == "Disfluent"]
matched_i_c_p = matched_i_c_p.append(interview_consonants_pws_disfluent)
matched_i_c_p.index = range(len(matched_i_c_p.index)) 

**2. Change Directory and Save**

In [183]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched


In [184]:
matched_r_v_p.to_excel("matchedVowels_reading.xlsx")
matched_r_c_p.to_excel("matchedConsonants_reading.xlsx")
matched_i_v_p.to_excel("matchedVowels_interview.xlsx")
matched_i_c_p.to_excel("matchedConsonants_interview.xlsx")

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/